## Loading libraries

In [ ]:
!pip install anndata scvi-tools numpy pandas scipy scanpy packaging==24.1

In [29]:
import anndata as ad
import scvi
import numpy as np
import pandas as pd
from scipy.stats import median_abs_deviation
import scanpy as sc
import os

C:\Users\athos\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h2>Reading Data - Situation 1: Building data from h5 file</h2>

In [57]:
adata = sc.read_10x_h5("Datasets/pbmc_10k_matrix_raw.h5")
adata.var_names_make_unique()

C:\Users\athos\anaconda3\Lib\site-packages\anndata\_core\anndata.py:1793: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\athos\anaconda3\Lib\site-packages\anndata\_core\anndata.py:1793: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


<h2>Reading Data - Situation 2: Building data from matrix-barcode-feature format</h2>

<p>Here, we build a Anndata object from gene (can be named features), barcodes, matrix (mtx) and metadata. The steps for building the object from these four files are:
<ul>
   <li>Getting the samples location.</li>
   <li>For each one, get the sample name - that is the name of the folder containing the files for that sample.</li>
   <li>Reading each sample folder, we read:</li>
    <ul>
       <li>Matrix - matrix.mtx.gz</li>
       <li>Barcodes/Observations/Cells - barcodes.tsv.gz</li>
       <li>Features/Genes - features.tsv.gz</li>
       <li>Metadata - meta.csv</li>
    </ul>
</ul>

This is just an example, as there are lots of different formats of storing scRNA-seq data.
</p>

In [ ]:
location = '/counts'

sample_names = []
for sample in os.listdir(location):
    if os.path.isdir(os.path.join(location, sample)):
        sample_names.append(sample)
print(sample_names)
        
sample_list = []

for name in sample_names:
    # Getting anndata (transposed to obs X vars)
    path = f'{location}/{name}/matrix.mtx.gz'
    sample = sc.read(path, cache=True).T
    
    # Getting obs
    path = f'{location}/{name}/barcodes.tsv.gz'
    obs = pd.read_csv(path, sep='\t', header=None, index_col=0)
    obs.index.name = 'barcode'
    sample.obs = obs

    # Getting vars
    path = f"{location}/{name}/features.tsv.gz"
    var = pd.read_table(path, sep='\t', header=None, index_col=1)
    var.index.name = 'genes'
    sample.var = var
    sample.var_names_make_unique(join="-")
    
    # Adding metadata
    sample.obs['Patient'] = name
    sample.obs['Condition'] = name[-1].upper()
    
    sample_list.append(sample)

## Normalization & highly variable genes (HVG)

Obs1 → We will use counts further to doublet removal. The purpose of saving it in layers is because the count matrix (adata.X) will be replaced by the normalizes counts. This differs scanpy from Seurat: Seurat always saves a copy of the processed matrix.
<br><br>
Obs2 → Normalize total makes a fixed total expression per cell of 1e4 (10,000) per gene. Log1p log transforms it with log(1+x), x being each cell-gene expression value. Log is in natural base (e).
<br><br>
Obs3 → Add a batch key only in situations where it is needed: when we gather many different datasets. You should put the column that would state this difference: if we have a "Author" column, we distinguish each dataset generating method by the name of the author of the dataset paper.

In [ ]:
sc.pp.filter_cells(adata, min_counts=500)
sc.pp.filter_cells(adata, min_genes=200)

# Obs1
adata.layers['counts']=adata.X.copy()

# Obs2
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Obs3
sc.pp.highly_variable_genes(adata, flavor='seurat_v3', n_top_genes=3000, 
                            layer='counts',subset=True, batch_key='Author')

## Removing doublets with SOLO (scVI)

<p>Doublets are cells that were sequenced together. Thus, they add bias to the further analysis. That said, in the majority of cases we should remove them:</p>

In [87]:
solo_df

,doublet,singlet
AAACCCAAGCGCCCAT-1,0.044233,0.955767
AAACCCAAGGTTCCGC-1,0.276011,0.723989
AAACCCACAGAGTTGG-1,0.017707,0.982293
AAACCCACAGGTATGG-1,0.529655,0.470345
AAACCCACATAGTCAC-1,0.109963,0.890037
...,...,...
TTTGTTGGTGTCATGT-1,0.025472,0.974528
TTTGTTGGTTTGAACC-1,0.009798,0.990202
TTTGTTGTCCAAGCCG-1,0.116983,0.883017
TTTGTTGTCTTACTGT-1,0.103846,0.896154


In [75]:
scvi.model.SCVI.setup_anndata(adata, layer='counts')
model = scvi.model.SCVI(adata)
model.train(max_epochs=100)
solo = scvi.external.SOLO.from_scvi_model(model)
solo.train()
solo_df = solo.predict()

adata.obs['doublet_pred'] = solo_df["doublet"] > solo_df["singlet"]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\athos\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 100/100: 100%|██████████| 100/100 [10:17<00:00,  6.72s/it, v_num=1, train_loss=978]  

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|██████████| 100/100 [10:17<00:00,  6.18s/it, v_num=1, train_loss=978]
INFO     Creating doublets, preparing SOLO model.                                                                  


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\athos\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
C:\Users\athos\anaconda3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 277/400:  69%|██████▉   | 277/400 [09:50<04:22,  2.13s/it, v_num=1, train_loss_step=0.309, train_loss_epoch=0.242]
Monitored metric validation_loss did not improve in the last 30 records. Best score: 0.228. Signaling Trainer to stop.


C:\Users\athos\anaconda3\Lib\site-packages\torch\utils\_contextlib.py:120: UserWarning: Prior to scvi-tools 1.1.3, `SOLO.predict` with `soft=True` (the default option) returned logits instead of probabilities. This behavior has since been corrected to return probabiltiies. The previous behavior can be replicated by passing in `return_logits=True`.
  return func(*args, **kwargs)


AttributeError: 'DataFrame' object has no attribute 'prediction'

In [89]:
adata.obs['doublet_pred'] = solo_df["doublet"] > solo_df["singlet"]
adata.obs['doublet_pred']

AAACCCAAGCGCCCAT-1    False
AAACCCAAGGTTCCGC-1    False
AAACCCACAGAGTTGG-1    False
AAACCCACAGGTATGG-1     True
AAACCCACATAGTCAC-1    False
                      ...  
TTTGTTGGTGTCATGT-1    False
TTTGTTGGTTTGAACC-1    False
TTTGTTGTCCAAGCCG-1    False
TTTGTTGTCTTACTGT-1    False
TTTGTTGTCTTCTAAC-1    False
Name: doublet_pred, Length: 11792, dtype: bool

In [105]:
print(adata.shape)
print(adata[~adata.obs.doublet_pred].shape)
print("Number of predicted doublets: ", adata.shape[0] - adata[~adata.obs.doublet_pred].shape[0])

(11792, 3000)
(10534, 3000)
Number of doublets:  1258


## QC

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')
adata.var['ribo'] = adata.var_names.str.startswith(('RPS','RPL'))
adata.var['hb'] = adata.var_names.str.startswith(("^HB[^(P)]"))

sc.pp.calculate_qc_metrics(
    adata, qc_vars=['mt','ribo','hb'], inplace=True, percent_top=[20], log1p=True
)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_ribo',
                     'pct_counts_hb', 'pct_counts_mt'], jitter=0.4, multi_panel=True)

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)

adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (
    adata.obs["pct_counts_mt"] > 8
)

adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

In [ ]:
adata.X = adata.layers['counts'].copy()
adata.write('raw_clean_adata.h5ad')

## scVI Integration

In [ ]:
scvi.model.SCVI.setup_anndata(adata, batch_key='Patient', layer='counts')

In [ ]:
arches_params = dict(
    use_layer_norm='both',
    use_batch_norm='none',
    encode_covariates=True,
    dropout_rate=0.2,
    n_layers=2,
)

vae = scvi.model.SCVI(adata, **arches_params)
vae.train()

In [ ]:
adata.obsm["X_scVI"] = vae.get_latent_representation() #dimensional reduction
adata.layers['scvi_normalized'] = vae.get_normalized_expression(library_size = 1e4)
sc.pp.neighbors(adata, use_rep="X_scVI")
sc.tl.umap(adata)

## Umap Visualization

In [ ]:
sc.pl.umap(adata, color = ['Condition', 'Patient'], frameon=False, ncols = 2)

In [ ]:
adata.write_h5ad("adataV1.h5ad")